In [1]:
import scipy

In [2]:
import numpy

In [3]:
import reportlab

In [4]:
from reportlab.graphics import renderPDF

In [5]:
from Bio.Seq import Seq

In [6]:
 from Bio.Alphabet.IUPAC import unambiguous_dna

In [7]:
import Bio

In [8]:
from Bio.Align.Applications import ClustalwCommandline

In [9]:
dir(Bio.Align.Applications)

['ClustalOmegaCommandline',
 'ClustalwCommandline',
 'DialignCommandline',
 'MSAProbsCommandline',
 'MafftCommandline',
 'MuscleCommandline',
 'PrankCommandline',
 'ProbconsCommandline',
 'TCoffeeCommandline',
 '_ClustalOmega',
 '_Clustalw',
 '_Dialign',
 '_MSAProbs',
 '_Mafft',
 '_Muscle',
 '_Prank',
 '_Probcons',
 '_TCoffee',
 '__all__',
 '__builtins__',
 '__doc__',
 '__file__',
 '__name__',
 '__package__',
 '__path__']

In [10]:
from Bio import AlignIO
#align = AlignIO.read("opuntia.aln", "clustal")

In [11]:
from Bio import Phylo
#tree = Phylo.read("opuntia.dnd", "newick")
#Phylo.draw_ascii(tree)

You're reading a fastq file, right? You're most probably reinventing the wheel - you could just use Biopython, it has tools for dealing with common biology file formats. For instance see this tutorial, for doing something with fastq files - it looks basically like this:

from Bio import SeqIO
for record in SeqIO.parse("SRR020192.fastq", "fastq"):
    # do something with record, using record.seq, record.id etc
More on biopython SeqRecord objects here.

Here is another biopython fastq-processing tutorial, including a variant for doing this faster using a lower-level library, like this:

from Bio.SeqIO.QualityIO import FastqGeneralIterator
for title, seq, qual in FastqGeneralIterator(open("untrimmed.fastq")):
    # do things with title,seq,qual values
There's also the HTSeq package, with more deep-sequencing-specific tools, which I actually use more often.

By the way, if you don't know about Biostar already, you could take a look - it's a StackExchange-format site specifically for bioinformatics.

In [12]:
# READING VCF files: use vcf

In [13]:
import vcf

In [14]:
vcf_reader = vcf.Reader(open('/Users/randalgoomer/Desktop/MM VCFs miSeq/MM348ccf-33889850.vcf', 'r'))

In [15]:
record = next(vcf_reader)
print record.POS
print record.ALT
print record.INFO
print record.var_type, record.var_subtype
print record.is_deletion, record.is_snp, record.is_indel
print record.num_called, record.num_het, record.nucl_diversity

43815071
[T]
{'FC': ['Noncoding'], 'GI': ['MPL'], 'DP': 562, 'TI': ['NM_005373']}
indel del
True False True
1 1 1.0


In [16]:
#Finally, record.samples is a list of dictionaries containing the parsed
#sample column and record.genotype is a way of looking up genotypes by sample name
record = next(vcf_reader)
for sample in record.samples:
    print sample['GT']

0/1


The genotypes are represented by Call objects, which have three 
attributes: the corresponding Record site, the sample name in sample 
    and a dictionary
    of call data in data:

In [17]:
print record.get_hets()

[Call(sample=MM348ccf, CallData(GT=0/1, GQ=61, AD=[814, 26], VF=0.031, NL=20, SB=-21.9886, GQX=61))]


In [18]:
call = record.genotype('MM348ccf')
print call.site
print call.data
print call.sample

Record(CHROM=chr1, POS=115258738, REF=C, ALT=[T])
CallData(GT=0/1, GQ=61, AD=[814, 26], VF=0.031, NL=20, SB=-21.9886, GQX=61)
MM348ccf


In [19]:
vcf_reader.metadata

OrderedDict([('fileformat', 'VCFv4.1'),
             ('annotator', ['MARS']),
             ('fileDate', '20160713'),
             ('source', ['CallSomaticVariants 3.2.3.0']),
             ('CallSomaticVariants_cmdline',
              ['" -B D:\\Illumina\\MiSeqAnalysis\\429ea50dd001441498daa64101304cdf\\Data\\Intensities\\BaseCalls\\Alignment -g [E:\\Genomes\\Homo_sapiens\\UCSC\\hg19\\Sequence\\WholeGenomeFASTA,] -f 0.01 -fo False -b 20 -q 100 -c 10 -s 0.5 -a 20 -F 20 -gVCF False -i true -r D:\\Illumina\\MiSeqAnalysis\\429ea50dd001441498daa64101304cdf"']),
             ('reference',
              'E:\\Genomes\\Homo_sapiens\\UCSC\\hg19\\Sequence\\WholeGenomeFASTA\\genome.fa')])

In [20]:
vcf_reader.samples

['MM348ccf']

In [21]:
vcf_reader.filters

OrderedDict([('LowVariantFreq',
              Filter(id='LowVariantFreq', desc='Low variant frequency < 0.01')),
             ('LowGQ', Filter(id='LowGQ', desc='GQ below < 30.00')),
             ('R8',
              Filter(id='R8', desc='IndelRepeatLength is greater than 8')),
             ('LowDP',
              Filter(id='LowDP', desc='Low coverage (DP tag), therefore no genotype called')),
             ('SB', Filter(id='SB', desc='Variant strand bias too high'))])

In [22]:
vcf_reader.infos.keys

<bound method OrderedDict.keys of OrderedDict([('TI', Info(id='TI', num=None, type='String', desc='Transcript ID', source=None, version=None)), ('GI', Info(id='GI', num=None, type='String', desc='Gene ID', source=None, version=None)), ('EXON', Info(id='EXON', num=0, type='Flag', desc='Exon Region', source=None, version=None)), ('FC', Info(id='FC', num=None, type='String', desc='Functional Consequence', source=None, version=None)), ('DP', Info(id='DP', num=1, type='Integer', desc='Total Depth', source=None, version=None))])>

ALT records are actually classes, so that you can interrogate them:

>>> reader = vcf.Reader(open('vcf/test/example-4.1-bnd.vcf'))
>>> _ = next(reader); row = next(reader)
>>> print row
Record(CHROM=1, POS=2, REF=T, ALT=[T[2:3[])
>>> bnd = row.ALT[0]
>>> print bnd.withinMainAssembly, bnd.orientation, bnd.remoteOrientation, bnd.connectingSequence
True False True T
The Reader supports retrieval of records within designated regions for files with tabix indexes via the fetch method. This requires the pysam module as a dependency. Pass in a chromosome, and, optionally, start and end coordinates, for the regions of interest:

>>> vcf_reader = vcf.Reader(filename='vcf/test/tb.vcf.gz')
>>> # fetch all records on chromosome 20 from base 1110696 through 1230237
>>> for record in vcf_reader.fetch('20', 1110695, 1230237):  
...     print record
Record(CHROM=20, POS=1110696, REF=A, ALT=[G, T])
Record(CHROM=20, POS=1230237, REF=T, ALT=[None])

In [23]:
import pysam

In [24]:
# fetch all records on chromosome 4 from base 11 through 20
vcf_reader.fetch('4', 10, 20)

IOError: index `/Users/randalgoomer/Desktop/MM VCFs miSeq/MM348ccf-33889850.vcf.tbi` not found

In [25]:
#The Writer class provides a way of writing a VCF file. Currently, you must specify a template Reader which provides the metadata:

vcf_reader = vcf.Reader(filename='/Users/randalgoomer/Desktop/MM VCFs miSeq/MM348ccf-33889850.vcf')
vcf_writer = vcf.Writer(open('/dev/null', 'w'), vcf_reader)
for record in vcf_reader:
    vcf_writer.write_record(record)